In [1]:
#===============================================================================================
# Huggingface와 ONNX 런타임을 이용한 동적 양자화 예시 3
# => 훈련된 bert 모델을 가지고, ONNX 모데로 만든 후, 테스트 하는 예시
# => 분류모델(NLI포함)이면 ORTModelForSequenceClassification 이용
#
#
#
## 참고 : https://huggingface.co/blog/optimum-inference
##
## 아래 패키지들을 설치해야 함
'''
!pip install datasets
!pip install optimum
!pip install optimum[onnxruntime]
!pip install optimum[onnxruntime-gpu]  #gpu 사용인 경우

'''    
#===============================================================================================

import numpy as np
import pandas as pd
import torch
import os
import torch.nn.functional as F
import sys

from myutils import seed_everything, GPU_info, mlogging
logger = mlogging(loggername="distilbertnlitest", logfilename="distilbertnlitest")
seed_everything(111)

logfilepath:../../log/bwdataset_2022-05-26.log
logfilepath:../../log/qnadataset_2022-05-26.log
logfilepath:distilbertnlitest_2022-05-26.log


In [2]:
# 1. ONNX 모델로 변환 
# => huggingface ORTModelForSequenceClassification 를 이용하여, ONNX 모델로 쉽게 저장할수 있다.(*단 여기서 ONNX 모델은 양자화된 모델은 아니고, 형식만 변경한 것임)
# => 양자화 하려면. 위에서 처럼 ORTQuantizer 이용해야 함.

from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer

model_checkpoint = "./distilbert-1"

# Load model from hub and export it through the ONNX format 
model = ORTModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=3,
    from_transformers=True
)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the exported model
onnx_path = './onnxfolder'
model.save_pretrained(onnx_path)
tokenizer.save_pretrained(onnx_path)

('./onnxfolder\\tokenizer_config.json',
 './onnxfolder\\special_tokens_map.json',
 './onnxfolder\\vocab.txt',
 './onnxfolder\\added_tokens.json',
 './onnxfolder\\tokenizer.json')

In [4]:
#2. 최적화 적용
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig

# optimization 최적화 실행
# optimization_config=99 enables all available graph optimisations(99이면 모든 것을 최적화 시킴)
optimization_config = OptimizationConfig(optimization_level=99)

optimizer = ORTOptimizer.from_pretrained(model_checkpoint, feature="sequence-classification")

optimizer.export(
    onnx_model_path='./onnxfolder/model.onnx',   # 앞에서 만든 ONNX 모델 경로
    onnx_optimized_model_output_path='./onnxfolder/model-optimized.onnx',  # 새롭게 만들 optimized 모델 경로
    optimization_config=optimization_config,
)



WindowsPath('onnxfolder/model-optimized.onnx')

In [5]:
# 3. 양자화 
from optimum.onnxruntime import ORTConfig, ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig


# 동적 양자화인 경우 is_static = False로 해야 함.
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)

# 분류 모델인 경우에는 feature="sequence-classification"
quantizer = ORTQuantizer.from_pretrained(model_checkpoint, feature="sequence-classification")

# ONNX 모델로 만들고 양자화 함
quantizer.export(
    onnx_model_path='./onnxfolder/model-optimized.onnx',   # optimized 모델 경로
    onnx_quantized_model_output_path="./onnxfolder/model-quantized.onnx",  
    quantization_config=qconfig,
)


WindowsPath('onnxfolder/model-quantized.onnx')

In [7]:
#============================================================
# 양자화 모델 불러옴
# => 양자화 모델은 model.eval() 하면 에러남.
#============================================================

from transformers import AutoTokenizer
from optimum.onnxruntime import ORTModelForFeatureExtraction, ORTModelForSequenceClassification

vocab_path = "./onnxfolder"
model_path = "./onnxfolder"

tokenizer = AutoTokenizer.from_pretrained(vocab_path)

# 분류모델이면, ORTModelForSequenceClassification
# 문장임베딩이면, ORTModelForFeatureExtraction 호출
model = ORTModelForSequenceClassification.from_pretrained(model_path, num_labels=3)

print(model)

In [ ]:
'''
#================================================================
# 기존 bert 모델 불러옴
#================================================================
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

vocab_path = "./distilbert-0331-TS-nli-0.1-10"
model_path = "./distilbert-0331-TS-nli-0.1-10"

tokenizer = DistilBertTokenizer.from_pretrained(vocab_path, do_lower_case=False)
model = DistilBertForSequenceClassification.from_pretrained(model_path, num_labels=3)
model.eval()
'''

In [8]:
# 평가 data loader 생성

from torch.utils.data import DataLoader, RandomSampler

from myutils import ClassificationDataset, KlueNLICorpus, data_collator, KorNLICorpus
#############################################################################
# 변수 설정
#############################################################################
max_seq_len = 128   # 글자 최대 토큰 길이 해당 토큰 길이 이상은 잘린다.
batch_size = 32        # 배치 사이즈(64면 GUP Memory 오류 나므로, 32 이하로 설정할것=>max_seq_length 를 줄이면, 64도 가능함)
cache = True   # 캐쉬파일 생성할거면 True로 (True이면 loding할때 캐쉬파일있어도 이용안함)
#############################################################################

# corpus 파일 설정
#corpus = KlueNLICorpus()
corpus = KorNLICorpus()

# 평가 dataset 생성
#file_fpath = '../../korpora/klue-nli/klue-nli-v1.1_dev.json'
file_fpath = './korpora/xnli.test.ko.tsv'
    
dataset = ClassificationDataset(file_fpath=file_fpath, max_seq_length=max_seq_len, tokenizer=tokenizer, corpus=corpus, overwrite_cache=cache)

# 평가 dataloader 생성
eval_loader = DataLoader(dataset, 
                          batch_size=batch_size, 
                          #shuffle=True, # dataset을 섞음
                          sampler=RandomSampler(dataset, replacement=False), #dataset을 랜덤하게 샘플링함
                          collate_fn=data_collator, # dataset을 tensor로 변환(예시 {'input_ids':tensor[0,1,2,3,1,], 'token_type_id:tensor[0,0,0,0,0], 'attention_mask:tensor[1,1,1,1,1], 'labels':tensor[5]}
                          num_workers=4)

print('eval_loader_len: {}'.format(len(eval_loader)))

2022-05-26 16:25:44,727 - bwpdataset - INFO - Creating features from dataset file at ./korpora\xnli.test.ko.tsv
INFO:bwpdataset:Creating features from dataset file at ./korpora\xnli.test.ko.tsv
2022-05-26 16:25:44,736 - bwpdataset - INFO - loading data... LOOKING AT ./korpora\xnli.test.ko.tsv
INFO:bwpdataset:loading data... LOOKING AT ./korpora\xnli.test.ko.tsv
2022-05-26 16:25:44,783 - bwpdataset - INFO - tokenize sentences, it could take a lot of time...
INFO:bwpdataset:tokenize sentences, it could take a lot of time...
2022-05-26 16:25:45,925 - bwpdataset - INFO - tokenize sentences [took 1.139 s]
INFO:bwpdataset:tokenize sentences [took 1.139 s]


  0%|          | 0/5010 [00:00<?, ?it/s]

2022-05-26 16:25:46,044 - bwpdataset - INFO - *** Example ***
INFO:bwpdataset:*** Example ***
2022-05-26 16:25:46,050 - bwpdataset - INFO - sentence A, B: 글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다. + 나는 그와 다시 이야기하지 않았다.
INFO:bwpdataset:sentence A, B: 글쎄, 나는 그것에 관해 생각조차 하지 않았지만, 나는 너무 좌절했고, 결국 그에게 다시 이야기하게 되었다. + 나는 그와 다시 이야기하지 않았다.
2022-05-26 16:25:46,057 - bwpdataset - INFO - tokens: [CLS] [UNK] , 나는 그것 ##에 관 ##해 생각 ##조 ##차 하지 않 ##았 ##지만 , 나는 너 ##무 좌절 ##했고 , 결국 그 ##에게 다시 이야기 ##하게 되었다 . [SEP] 나는 그 ##와 다시 이야기 ##하지 않았다 . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [

eval_loader_len: 157


In [9]:
import time
from tqdm.notebook import tqdm
import torch.nn.functional as F

logger.info(f"=== model: {model_path} ===")
logger.info(f"eval_file : {file_fpath}")
#logger.info(f"num_parameters: {model.num_parameters()}")

# 평가 시작
#model.eval()

total_loss = 0
total_len = 0
total_correct = 0

start = time.time()
logger.info(f'---------------------------------------------------------')

for data in tqdm(eval_loader):
    # 입력 값 설정
    # =>**distilbert에는 token_type_ids가 없다
    labels = data['labels']
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
 
    # 손실률 계산하는 부분은 no_grade 시켜서, 계산량을 줄임.
    # => torch.no_grad()는 gradient을 계산하는 autograd engine를 비활성화 하여 
    # 필요한 메모리를 줄이고, 연산속도를 증가시키는 역활을 함
    with torch.no_grad():
        # 모델 실행
        outputs = model(input_ids=input_ids, 
                       attention_mask=attention_mask,
                       labels=labels)
    
        # 출력값 loss,logits를 outputs에서 얻어옴
        loss = outputs.loss
        logits = outputs.logits

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)

logger.info(f"eval-accuracy: {total_correct / total_len}")
logger.info(f'---------------------------------------------------------')
logger.info(f'=== 처리시간: {time.time() - start:.3f} 초 ===')
logger.info(f'-END-\n')

2022-05-26 16:25:51,624 - distilbertnlitest - INFO - === model: ./onnxfolder ===
INFO:distilbertnlitest:=== model: ./onnxfolder ===
2022-05-26 16:25:51,638 - distilbertnlitest - INFO - eval_file : ./korpora/xnli.test.ko.tsv
INFO:distilbertnlitest:eval_file : ./korpora/xnli.test.ko.tsv
2022-05-26 16:25:51,640 - distilbertnlitest - INFO - ---------------------------------------------------------
INFO:distilbertnlitest:---------------------------------------------------------


  0%|          | 0/157 [00:00<?, ?it/s]

C:\Users\bong9\AppData\Local\Temp/ipykernel_13492/3622772955.py:39: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  pred = torch.argmax(F.softmax(logits), dim=1)
2022-05-26 16:41:07,911 - distilbertnlitest - INFO - eval-accuracy: 0.7289421157684631
INFO:distilbertnlitest:eval-accuracy: 0.7289421157684631
2022-05-26 16:41:07,916 - distilbertnlitest - INFO - ---------------------------------------------------------
INFO:distilbertnlitest:---------------------------------------------------------
2022-05-26 16:41:07,925 - distilbertnlitest - INFO - === 처리시간: 916.284 초 ===
INFO:distilbertnlitest:=== 처리시간: 916.284 초 ===
2022-05-26 16:41:07,930 - distilbertnlitest - INFO - -END-

INFO:distilbertnlitest:-END-



In [ ]:

# 모델 추론 테스트 

import torch.nn.functional as F

inputs = tokenizer("날씨가 흐리다", "비가 내린다",return_tensors="pt")
outputs = model(**inputs)

print(outputs)
logits = outputs.logits
print(logits)

pred = torch.argmax(F.softmax(logits), dim=1)
print(pred)

if pred == 0:
    print('참(수반:entailment)')
elif pred == 1:
    print('거짓(모순:contradiction)')
elif pred == 2:
     print('모름(중립:neutral)')


In [ ]:
'''
# 참고
# => huggingface ORTModelForSequenceClassification 를 이용하여, ONNX 모델로 쉽게 저장할수 있다.(*단 여기서 ONNX 모델은 양자화된 모델은 아니고, 형식만 변경한 것임)
# => 양자화 하려면. 위에서 처럼 ORTQuantizer 이용해야 함.

from optimum.onnxruntime import ORTModelForSequenceClassification

model_checkpoint = "./distilbert-0331-TS-nli-0.1-10"

# Load model from hub and export it through the ONNX format 
model = ORTModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=3,
    from_transformers=True
)

# Save the exported model
model.save_pretrained("./onnxfolder")
'''

In [ ]:
'''
# onnx 모델 구조 로딩 해봄.
# => 맨 뒤에 return %last_hidden_state 리턴되면 => ORTModelForFeatureExtraction 사용 가능
import onnx
model = onnx.load("./distilbert-nli/model.onnx")
onnx.checker.check_model(model)
print(onnx.helper.printable_graph(model.graph))
'''